# SMU 7331 Data Mining Project 1
##### Authors: Shon Mohsin, Heber Nielsen, Jose Torres, Lokesh Maganti

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix, parallel_coordinates
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

%matplotlib inline

## Business Understanding:

Our selected data set was chosen because we had an interest in the subject matter written about in the following paper: _["Effects of earthquake on perinatal outcomes: A Chilean register-based study"](https://github.com/ShonTM/DataMiningProject1/blob/master/chile%20earthquake%20plos%20one%20paper.pdf)_  by Yasna K. Palmeiro-Silva, et al. The paper studied whether the February 27, 2010 earthquake in Chile affected the perinatal outcomes of Chilean pregnant women. 

For this project, we visualize the dataset used and use exploratory data analysis to identify possible interpretations of the visualizations. We create summary statistics to derive high-level opportunities for further analysis. 

Useful knowledge represents differences in the visualizations that identify changes in the babies' fetal growth and early adaptation to neonatal life associated with exposure to the earthquake.

We can measure effectiveness of a good prediction algorithm by examining model performance metrics such as accuracy, precision, or recall (if classification model), or R^2/AIC/BIC values (regression model).  One useful target variable from this data set is earthquake xposure (Exposed).

### Data Collection Process:

The following description of the data gathering process is from the paper describing the data quality:

All demographic and clinical information was abstracted and recorded by professional workers (medical doctors, midwives, and nurses). The data abstracted included: date of delivery, gestational age at delivery (in weeks), maternal age (in years), parity of mother, the place of residence, the gender of newborns, birth weight (in grams), length (in centimeters), head circumference (in centimeters), Apgar at 1 and 5 minutes, if small for gestational age (SGA) (defined as newborn birth weight <10th percentile for gestational age), ponderal index (weight/height3), preterm delivery (birth between 34–37 weeks). Gender of newborns, location of residence, gestational age (either in weeks or days), parity and maternal age were considered by the study authors as effect modifiers; location of residence was related to three categories of average annual income: low income (500,000 to 1 million Chilean pesos), middle income (1 to 1.5 million Chilean pesos) and high income (more than 1.5 million Chilean pesos).  

### Data Preparation

The columns are converted to their english values as described in the Data Overview section. The conversion is done directly to the csv file using excel and find and replace functions in python for the class variables. 

## Data Features ##

| Feature Name (Original)       | Feature Name (English) | Description               | Variable Type                |
| -----------------------------:| ----------------------:|--------------------------:|-----------------------------:|
| id_clinica                    | Clinic_ID              | ID# of the medical clinic | numeric                      |
| id_excel                      | Excel_File_ID          | ID# in original source    |                              |
| dia                           | Day                    | Day in birth month        | numeric                      |
| mes                           | Month                  | Birth month               | numeric                      |
| ano                           | Year                   | Birth year                | numeric                      |
| sex                           | Sex                    | Birth sex (Mujer/Hombre)  | numeric                      |
| peso                          | Weight                 | Birth weight in grams| numeric                           |
| talla                         | Length                 | Birth length in cm        | numeric                      |
| cc                            | Head_circ              | Head circumference in cm (eyebrow level) | numeric       |
| apgar                         | Apgar_1                | Apgar score 1 minute after birth | numeric               |
| apgar5                        | Apgar_5                | Apgar score 5 minutes after birth | numeric              |
| comuna                        | Municipality           | Avg. annual income in Chilean Pesos | numeric            |
| aeg                           | wgt_for_age            | Weight relative to gestational age | Categorical         |
| eg                            | Gest_age               | Gestational age at birth in weeks  | numeric             |
| trim_exp                      | Trimester              | Trimester on February 27 of either study year| numeric   |
| bajo_peso                     | Low_bthwgt             | Indicator for weight under 2500 grams | Categorical      |
| pretermino                    | Premature              | Indicator for gestation period less than 37 weeks | Categorical  |
| edad_mama                     | Maternal_Age           | Age of mother in years    | numeric                      |
| paridad                       | Parity                 | Number of previous live births by same mother | numeric  |
| trim_exp_g                    | Trim_study             | Trimester combined with study year (1-6)   | Categorical |
| pi                            | Pondural Index         | Calculation representing fetal growth | numeric          |
| exposed                       | Exposed                | Indicator of control or earthquake group | Categorical   |

One additional comment on APGAR scores.  APGAR is an acronym for the categories: Activity, Pulse, Grimace, Appearance, and Respiration.  It summarizes how well the newborn is making the transition to life outside the uterus.  Points are assigned to each each category based on level of activity observed, and range from 0 to 2.  The sum of all points (maximum score = 10) represents the APGAR score.

In [2]:
url = 'https://raw.githubusercontent.com/ShonTM/DataMiningProject1/master/Chilean%20Earthquake%20Perinatal%20Outcome_english_translation.csv' 
df=pd.read_csv(url)

## Data Quality

The discussion and cells below provide commentary on the quality of this data set.

### Missing Values
`Clinic_ID` and `Excel_File_ID` are dropped because they are not needed for analysis. Furthermore, the two `NaN` values in `Parity` are inputed with the most commonly observed value of 0.  Since 0 is the mode of `Parity`, we impute the two NaNs with 0.

### Duplicates

We found `0` duplicates in this data set (see next cell).

### Outliers

We identified 3 potential outliers based on the scatter plot below of `Weight` vs `Ponderal_index`.  These are newborns whose lengths were much smaller than typical given their weight (more than 4 standard deviations away from the mean).

In [3]:
#Data Cleanup - Dropping unnecessary columns, replacing NaNs, and fixing column names
print('Dropping columns Clinic_ID and Excel_File_ID...')
print('Replacing NaN values in \'Parity\' with 0...')
print('Renaming Lenght column to Length..')

#drop "Clinic_ID" and "Excel_File_ID" from dataframe
df.drop('Clinic_ID', axis=1, inplace=True, errors='ignore')
df.drop('Excel_File_ID', axis=1, inplace=True, errors='ignore')


#identify records with missing values
#display(df.iloc[df['Parity'].isna().get_values(),:].transpose() \
#           .style.highlight_null(null_color='red'))

#replace NaN in Parity with 0, which is the most common label
df['Parity'].fillna(0, inplace=True)


#rename length column
df.rename(columns={'Lenght': 'Length'}, inplace=True)

Dropping columns Clinic_ID and Excel_File_ID...
Replacing NaN values in 'Parity' with 0...
Renaming Lenght column to Length..


In [4]:
print('There are {} duplicated records.'.format(df.duplicated().sum()))

There are 0 duplicated records.


# Data Visualization and Visual Analysis #

Categorical variables identified: `Sex`, `Municipality`, `wgt_for_age`, `Trimester`, `Low_birthwgt`, `Premature`, `Trim_study`, `Parity`, `Exposed`, `Apgar_1`, `Apgar_5`, `Gest_age`

Continuous variables identified: `Weight`, `Length`, `Head_circ`, `Maternal_age`, `Ponderal_index`

In [ ]:
#ignore some columns
columns_ignore = ['Day', 'Month', 'Year']
df_lite = df.drop(columns_ignore, axis=1)

#map some categorical variables to work with parallel coordinate plots and have a more natural ordering
df_lite['Sex'] = df_lite['Sex'].map({'female': 0, 'male': 1})
df_lite['Premature'] = df_lite['Premature'].map({'Premature': 1, 'Not premature': 0})
df_lite['Trim_study'] = df_lite['Trim_study'].map({'First2009': '1', 'Second2009': '2', 'Third2009': '3', 
                          'First2010': '4', 'Second2010': '5', 'Third2010': '6' })

#get list of categorical variables
categories = list(df_lite.columns[df_lite.dtypes == 'object']) \
                    + ['Parity', 'Apgar_1', 'Apgar_5', 'Exposed', 'Sex', 'Premature', 'Trim_study', 'Gest_age']
continuous = [col for col in df_lite.columns if col not in categories]

#confirm list of categorical variables
print('Categorical variables: {}\n'.format(categories))
print('Continuous variables: {}\n'.format(continuous))

#change data type for categorical variables to pandas 'category' type
df_lite[categories] = df_lite[categories].astype('category')

Categorical variables: ['Municipality', 'wgt_for_age', 'Trimester', 'Low_birthwgt', 'Trim_study', 'Parity', 'Apgar_1', 'Apgar_5', 'Exposed', 'Sex', 'Premature', 'Trim_study', 'Gest_age']

Continuous variables: ['Weight', 'Length', 'Head_circ', 'Maternal_age', 'Ponderal_index']



## Summary Statistics and Commentary ##

It is important to look at measurements of centrality and dispersion, such as mean and standard deviation, to help identify distributions and that observed values match prior expectations for those variables.

### Main Observations ###

Observations for continuous variables:
+ `weight` has much larger values (due to small unit of measure) as well as a larger range than the other continuous variables.  This should be normalized before fitting any statistical models that use distance-based measurements for error.

+ In the scatter plot matrix, there is evidence of symmetry in all continuous variables

Observations for categorical variables:
+ Most categorical variables have only a few categories (2 or 3)

In [ ]:
print('Summary Statistics for Continuous Features:')
display(df_lite[continuous].describe())
print('Summary Statistics for Categorical Features:')
display(df_lite[categories].describe())

### Correlation ###
- A few continuous variables appear approximately Normally distributed (e.g. `weight`, and `head_circ`), and all appear to be symmetric

- The correlation scores in the data frame below show strongest linear correlation for the following pairs:
 - corr(`weight`,`length`) = .72, 
 - corr(`weight`, `head_circ`) = .61
 - corr(`length`, `head_circ`) = .49
 - corr(`weight`, `ponderal_index`) = .49
 
We expect these strong correlations because this is known information. We can confirm that a higher weight shows a strong relationship to the length of the baby, along with the size of the head circumference. Overall, the correlations do not reveal any surprising relationships. 

In [ ]:
#https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html
display(df_lite.corr().style.background_gradient('coolwarm'))

In [ ]:
plt.style.use('default')
ax = scatter_matrix(df_lite,figsize=(10, 10))

If we look a little closer at correlation using bivariate contour plots, we see that there are a few cases with multiple density maxima, such as for `weight vs Head_circ` and `Head_circ` vs `Ponderal_index`. (Density maxima define a point process in which the fundamental quantity is the set of positions which are local maxima of the density field). These observations suggest looking more extensively for further hypothesis testing.

In [ ]:
#Show how balanced the counts are across categorical variables
fig, ax = plt.subplots(nrows=5, ncols=5, figsize=(20,20))
plt.style.use('ggplot')
#sns.set(style="white")
plt.suptitle('Number of Exposed and Non-Exposed Newborns')

for i in range(len(continuous)):
    for j in range(i+1, len(continuous)):
        sns.kdeplot(data=df_lite[continuous].iloc[:,i], 
                            data2=df_lite[continuous].iloc[:,j], ax=ax[i, j], shade_lowest=False, cmap='Blues')
        ax[i,j].set_title('{} vs {}'.format(continuous[i],continuous[j]))

#adjust tight_layout to avoid title clipping
fig.tight_layout(rect=[0, 0, 1, .94])

for i in range(5):
    for j in range(i+1):
        ax[i,j].set_visible(False)

plt.show()

### Revisiting Outliers ###

There are 3 outliers seen in the `weight` vs `Ponderal_index` scatterplot.  These are newborns whose lengths were much smaller than would be predicted given their weight (more than 4 standard deviations away from the mean).  Since `Ponderal_index` is inversely proportional to Length^3, it is high for these 3 unusually short babies. Since any genetic-related abnormalities are not captured in the data, we do not drop the outliers as there may be several factors that may be causing these outlier anomalies, which may be observed with a larger dataset. 

These potential outliers are identified in the plot below (in blue).

In [ ]:
x = df_lite['Ponderal_index']
y = df_lite['Weight']
c = df_lite['Ponderal_index'] >= 4.0

plt.figure(figsize=(5,5))
plt.style.use('ggplot')

ax = sns.scatterplot(x,y,hue=c)
ax.legend().set_visible(False)
ax.set_title('Outlier Identification')
plt.show()

### Statistical Comparisons ###

We divide our buckets by the binary 'Exposed' column to see if there are visually discernable observations that are evident between infants that were born before the earthquake or after the earthquake. 

Using the continuous variables, we examine means, medians, standard deviations, quartiles and ranges by exposure group, as well as the differences in means.  We see that the % change in the mean of each variable for Exposed babies is a slight decrease for the newborn measurements, and an increase in maternal age and ponderal index. 

In [ ]:
#mean values of target variables grouped by exposure to earthquake
summary_df = df_lite.groupby(by='Exposed').agg([np.mean, np.median, np.std]).transpose()
summary_df.columns = ['Not Exposed', 'Exposed']
summary_df['Difference'] = summary_df['Exposed'] - summary_df['Not Exposed']
summary_df['Difference %'] = summary_df['Difference'] / summary_df['Not Exposed'] * 100
summary_df.reset_index(inplace=True)
summary_df.columns = ['Variable', 'Statistic', 'Not Exposed', 'Exposed', 'Difference', 'Difference %']
summary_df

When we look at the distribution of newborns for each category in `Sex`, `Wealth`, `Trimester`, and `Parity` variables, we see the non-exposed (red) and exposed (blue) classes are **balanced** across all categories of all variables.  

These plots are interpreted by first picking a categorical variable, say Sex, then pick a category, say Female.  Of all Females, the red bar represents the number that were exposed.  The blue bar is the number that were not exposed.

Similar bar heights confirm the proportion of exposed newborns is similar to the proportion of non-exposed newborns. Thus, statistics computed for each category for exposed babies and non-exposed babies have a similar number of observations. Further anaylysis is needed to see if the differences observed between exposed and non-exposed are statistically significant. 

In [ ]:
#Show how balanced the counts are across categorical variables
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(15,10))
plt.style.use('ggplot')
plt.suptitle('Number of Exposed and Non-Exposed Newborns')

#plot each cross tabulation (% is relative to total newborns in a variable category)
independent_features = ['Sex', 'Municipality', 'Trimester', 'Parity', 'Premature']
for idx in range(len(independent_features)):
    row = int(idx / 3)
    col = int(idx % 3)
    pd.crosstab(index=df_lite[independent_features[idx]],columns=df_lite['Exposed'],
                colnames=['Stress Exposure']).plot(kind='bar', ax=ax[row,col])
    ax[row,col].set_ylabel('Count')
    ax[row,col].set_title(independent_features[idx])
    ax[row,col].set_xlabel('')

#adjust tight_layout to avoid title clipping
fig.tight_layout(rect=[0, 0, 1, .94])
ax[1,2].set_visible(False)

plt.show()

The distribution plots below show the continuous variables in a little more detail than the correlogram above.  `Weight` and `Ponderal_index` have the most bell-shaped curve. An interesting observation is the `Head_circ`distribution which shows a double peak with smaller peaks on each side. Differences in `Head_circ` are known to reflect difference in brain growth. Further analysis is warranted to discern any meaninful relationship to explain the peaks and valleys. 

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(12,7))
plt.style.use('ggplot')
plt.suptitle('Distribution Plots with Kernel Smoothing')

#plot each cross tabulation (% is relative to total newborns in a variable category)
independent_features = continuous
for idx in range(len(independent_features)):
    row = int(idx / 3)
    col = int(idx % 3)
    sns.distplot(df_lite[independent_features].iloc[:,idx],
                               hist=True,kde=True,color='darkblue',
                               hist_kws={'edgecolor':'black'},kde_kws={'linewidth':4}, ax=ax[row,col])
    ax[row,col].set_title(independent_features[idx])
    ax[row,col].set_xlabel('')

ax[1,2].set_visible(False)
#df_lite[continuous].apply(lambda x: plothist(x), axis=0)

To visualize the parallel coordinates, we changed the scale to normalize values for the dataset. 

From the output, we see that there is a stonger tendency for babies born after exposure to be mildly premature compared to babies born before exposure. This is interesting as we observe no other reason for this anomaly. However, causation does not equate to correlation, and further analysis must be pursued to determine if the exposure can be an explanation of this difference.

In [ ]:
# Plot of parallel coordinates. 

plt.close('all')
plt.figure(figsize=(10,10))

df_normalized = (df_lite[continuous] - df_lite[continuous].min()) / (df_lite[continuous].max() - df_lite[continuous].min())
df_normalized['Exposed'] = df_lite['Exposed']
df_normalized['Trim_study'] = df_lite['Trim_study']
df_normalized['Premature'] = df_lite['Premature']
#df_normalized['Sex'] = df_lite['Sex'].map({'female': 0, 'male': 1})
df_normalized['Apgar_5'] = df_lite['Apgar_5']
parallel_coordinates(df_normalized.loc[:,['Exposed', 'Apgar_5', 'Maternal_age', 'Premature']], 'Exposed',color=[[1,0,0,1],[0,1,0,.01]])

We compare box plots to understand the differences in weight categorized by trimester and sex.  Observing the box plot, we see two patterns:
+ Exposure in the second trimester produces more outliers in weight.
+ Exposed males tend to weigh less
+ Exposed premature babies appearing to weigh less than non-exposed premature babies

Next, we compare box plots to understand the differences in maternal age by trimester, baby's sex, and premature status.  Observing the box plot, we see two patterns:

+ Exposed babies tend to have older mothers than non-exposed babies
+ Premature exposed babies have a higher proportion of younger mothers than non-premature exposed babies

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(16,16))
plt.style.use('ggplot')
plt.suptitle('Distribution Plots with Kernel Smoothing')

independent_features = ['Trimester', 'Sex', 'Premature']
for idx in range(len(independent_features)):
    row = int(idx / 2)
    col = int(idx % 2)
    sns.boxplot(x=independent_features[idx], y='Weight', hue='Exposed', data=df_lite, ax=ax[row,col])
    ax[row,col].set_title(independent_features[idx])
    ax[row,col].set_xlabel('')
    
ax[0,1].set_xticklabels(labels = ['Female','Male'])
ax[1,0].set_xticklabels(labels = ['Not Premature','Premature'])
ax[1,1].set_visible(False)
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(16,16))
plt.style.use('ggplot')
plt.suptitle('Distribution Plots with Kernel Smoothing')

#plot each cross tabulation (% is relative to total newborns in a variable category)
independent_features = ['Trimester', 'Sex', 'Premature']
for idx in range(len(independent_features)):
    row = int(idx / 2)
    col = int(idx % 2)
    sns.boxplot(x=independent_features[idx], y='Maternal_age', hue='Exposed', data=df_lite, ax=ax[row,col])
    ax[row,col].set_title(independent_features[idx])
    ax[row,col].set_xlabel('')
    
ax[0,1].set_xticklabels(labels = ['Female','Male'])
ax[1,0].set_xticklabels(labels = ['Not Premature','Premature'])
ax[1,1].set_visible(False)
plt.show()

## Additional Features ##

Infant weight and length are used as inputs for the Ponderal index score.  
Other potential new features could be:
+ Discretizing some of the continuous variables into categories with larger differences (e.g. small/mid/large length)
+ Convert the APGAR scores into the meaningful categories that the numbers represent:
 + 0-3 = Severely depressed infant
 + 4-6 = Moderately depressed infant
 + or 7-10 = Excellent condition


## Principal Components Visualizations ## 

In [ ]:
###Redo the model, ask for 3 component model
#features = ['Sex', 'Head_circ', 'apgar_1', 'Apgar_5', 'Municipality', 'Wgt_for_age', 'Gest_age', 
#            'Trimester', 'Low_birthwgt', 'Premature', 'Maternal_age', 'Parity', 'Ponderal_index']
#extra_features = ['Weight', 'Length']

X = StandardScaler().fit_transform(df_lite[continuous].values)
y = df_lite[['Exposed']].values

pca = PCA().fit(X)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

#PCA Projection to smaller number of variables
#I chose 2 components because only a 2-dimensional plot is being done.

pca = PCA(n_components=2)
X_prin = pca.fit_transform(X)
principalDf = pd.DataFrame(data = np.concatenate((X_prin,y), axis=1),
                          columns = ['principal component 1', 'principal component 2', 'Exposed'])

In [ ]:
##Visualize the Projection
#Use a PcA projection to 2D to visualize the entire data set.
#Plot different classes using different colors or shapes.
#Do the classes seem well-seperated from each other?

fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 

ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 Component PCA', fontsize = 20)


sns.scatterplot(x=principalDf.iloc[:,0], y=principalDf.iloc[:,1], hue=principalDf.iloc[:,2])
plt.show()

### Modeling 

### Evaluation

### Deployment